Requires SkyTools: `conda install skytools`

In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import skytools as st

Set paths on NERSC:

In [ ]:
fg_path = f"/global/cfs/cdirs/cmb/gsharing/panexp_v1_planck/galactic_foregrounds_mediumcomplexity/"
npipe_path = f"/global/cfs/cdirs/cmb/data/planck2020/npipe/npipe6v20/"

In [ ]:
import os

if not os.path.exists("HFI_SkyMap_353_2048_R4.00_full.fits"):
    !wget "https://portal.nersc.gov/cfs/cmb/planck2020/frequency_maps/Single-frequency/HFI_SkyMap_353_2048_R4.00_full.fits"

Read NPIPE map, create a mask from the polarized intensity.

In [ ]:
hfi_npipe = hp.read_map(f"HFI_SkyMap_353_2048_R4.00_full.fits", field=(0, 1, 2)) * 1e6
polI_npipe = np.sqrt(hfi_npipe[1] ** 2 + hfi_npipe[2] ** 2)
mask_G = st.intensity_mask(hp.get_nside(hfi_npipe), polI_npipe, 3.0, smooth_in_deg=1.0)

mask = st.mask_udgrade(mask_G, 128)

In [ ]:
import pysm3
from pysm3 import units as u

In [ ]:
freq = 353
lmax = 100
hfi_npp = st.change_resolution(
    hfi_npipe * mask_G,
    nside_out=128,
    lmax_sht=lmax,
    mode="iqu",
    fwhm_in=5.0,
    fwhm_out=120.0,
)
hfi_npp *= mask

In [ ]:
from astropy.table import QTable

bandpass_table = QTable.read("../instrument_model/bandpass_353.tbl", format="ascii")
print(bandpass_table)

In [ ]:
bandpasses = [
    ("353 GHz", 353 * u.GHz, None),
    ("350 GHz", 350 * u.GHz, None),
    ("356 GHz", 356 * u.GHz, None),
    (
        "RIMO * nu^2",
        bandpass_table["bandpass_frequency"],
        bandpass_table["bandpass_weight"],
    ),
    (
        "RIMO",
        bandpass_table["bandpass_frequency"],
        bandpass_table["bandpass_weight"] / bandpass_table["bandpass_frequency"] ** 2,
    ),
]

In [ ]:
polI_npp = np.sqrt(hfi_npp[1] ** 2 + hfi_npp[2] ** 2)

mask = st.intensity_mask(128, polI_npp, 3.0, smooth_in_deg=1.0)

In [ ]:
sky = pysm3.Sky(nside=2048, preset_strings=["d10"], output_unit=u.uK_CMB)

lmax = 100
Bmode_sim = {}
for bandpass_name, freq, weights in bandpasses:

    print(bandpass_name)
    pysm_map = sky.get_emission(freq, weights)
    hfi_sim = (
        st.change_resolution(
            pysm_map * mask_G,
            nside_out=128,
            lmax_sht=lmax,
            mode="iqu",
            fwhm_in=5.0,
            fwhm_out=120.0,
        )
        * mask
    )
    Bmode_sim[bandpass_name] = st.iqu2teb(hfi_sim * mask, mode="b", lmax_sht=lmax)

In [ ]:
Bmode_npp = st.iqu2teb(hfi_npp * mask, mode="b", lmax_sht=lmax)

Scatter plot for B-modes:

In [ ]:
x = np.arange(-700, 700)
for bandpass_name in Bmode_sim.keys():
    plt.plot(
        Bmode_npp[0, mask > 0.5],
        Bmode_sim[bandpass_name][0, mask > 0.5],
        "o",
        alpha=0.33,
        ms=1.0,
        label=bandpass_name,
    )
plt.plot(x, x, "k-")
plt.legend()
plt.xlabel("NPIPE 353 GHz B-mode map")
plt.ylabel("PySM d10 353 GHz B-mode map")

In [ ]:
plt.figure(figsize=(10, 5))
x = np.arange(-700, 700)
for bandpass_name in Bmode_sim.keys():
    if bandpass_name in ["353 GHz", "356 GHz"]:

        plt.plot(
            Bmode_npp[0, mask > 0.5],
            Bmode_sim[bandpass_name][0, mask > 0.5],
            "o",
            alpha=0.33,
            ms=1.0,
            label=bandpass_name,
        )
plt.plot(x, x, "k-")
plt.legend()
plt.xlabel("NPIPE 353 GHz B-mode map")
plt.ylabel("PySM d10 353 GHz B-mode map")

In [ ]:
plt.figure(figsize=(10, 5))
x = np.arange(-700, 700)
for bandpass_name in Bmode_sim.keys():
    if bandpass_name in ["RIMO * nu^2", "RIMO"]:

        plt.plot(
            Bmode_npp[0, mask > 0.5],
            Bmode_sim[bandpass_name][0, mask > 0.5],
            "o",
            alpha=0.33,
            ms=1.0,
            label=bandpass_name,
        )
plt.plot(x, x, "k-")
plt.legend()
plt.xlabel("NPIPE 353 GHz B-mode map")
plt.ylabel("PySM d10 353 GHz B-mode map")

In [ ]:
from numpy.polynomial.polynomial import Polynomial
import pandas as pd

slopes = {}
for bandpass_name in Bmode_sim.keys():
    mask_indices = mask > 0.5
    x_data = Bmode_npp[0, mask_indices]
    y_data = Bmode_sim[bandpass_name][0, mask_indices]

    # Fit a linear polynomial (degree 1)
    p = Polynomial.fit(x_data, y_data, deg=1)
    slopes[bandpass_name] = p.convert().coef[1]  # Extract the slope

# Convert the slopes dictionary to a DataFrame
slopes_df = pd.DataFrame(list(slopes.items()), columns=["Bandpass", "Slope"])

# Display the DataFrame
print(slopes_df)